In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
import dgl
sys.path.append('.')

from glob import glob
from configs import *
from tqdm import tqdm
from utils import *
import numpy as np
# from dataset import *k
from days import *
from time import time
tqdm.monitor_interval = 0
import torch
import json
from time import time
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

In [11]:
groups = [
    
#     '/beegfs/ua349/logs/slurm_1353164.out',
#     ['Linear', '/beegfs/ua349/logs/linear', [2, 3, 4, 5]],
#     ['GCN', '/beegfs/ua349/logs/gcn', [2, 3, 4, 5]],
#     ['MXRNN', '/beegfs/ua349/logs/mxrnn', [2, 3, 4, 5]],
#     ['MPRNN', '/beegfs/ua349/logs/lgcn', [2, 3, 4]],
    ['ASTGCN', '/beegfs/ua349/logs/astgcn', [2, 3, 4, 5]],
]

In [12]:
delim = ' '
for name, folder, hops in groups:
    hop_avg = []
    for hop in range(1, 6):
        if hop in hops:
            avg = []
            for logfl in glob('%s/*n%d*' % (folder, hop)):
#                 print(logfl)
                with open(logfl) as fl:
                    res = json.load(fl)
                hist, best_eval = res
                avg.append(np.sqrt(best_eval))
            avg = np.mean(avg)
            hop_avg.append(avg)
        else:
            hop_avg.append(None)
    out = delim.join([
        name,
        delim.join(['%.4f' % avg if avg is not None else '-' for avg in hop_avg]),
    ])
    print(out)

ASTGCN - 0.1936 0.1922 0.1888 0.1867
